In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!apt install libspatialindex-dev
!pip install rtree
!pip install geopandas
!pip install pygeos
!pip install utm
!pip install alphashape
!pip install osmnx

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libspatialindex-c6 libspatialindex6
The following NEW packages will be installed:
  libspatialindex-c6 libspatialindex-dev libspatialindex6
0 upgraded, 3 newly installed, 0 to remove and 30 not upgraded.
Need to get 319 kB of archives.
After this operation, 1,416 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libspatialindex6 amd64 1.9.3-2 [247 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libspatialindex-c6 amd64 1.9.3-2 [55.8 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libspatialindex-dev amd64 1.9.3-2 [16.0 kB]
Fetched 319 kB in 1s (474 kB/s)
Selecting previously unselected package libspatialindex6:amd64.
(Reading database ... 121658 files and directories currently installed.)
Preparing to unpack .../libspatialindex6_1.9.3-2_amd64.deb ...
Unp

In [ ]:
import geopandas as gpd
building = gpd.GeoDataFrame.from_file('/content/drive/MyDrive/mir/final/2/OSM/finalpoly_after_buf.shp')
pois=gpd.GeoDataFrame.from_file('/content/drive/MyDrive/mir/final/2/OSM/All_poi_after_buf.shp')

/usr/local/lib/python3.10/dist-packages/geopandas/_compat.py:124: UserWarning: The Shapely GEOS version (3.11.2-CAPI-1.17.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
<ipython-input-3-27c4322e1701>:1: DeprecationWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In the next release, GeoPandas will switch to using Shapely by default, even if PyGEOS is installed. If you only have PyGEOS installed to get speed-ups, this switch should be smooth. However, if you are using PyGEOS directly (calling Py

In [ ]:
!pip install trackintel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.9/154.9 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.0/305.0 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.3/401.3 kB 12.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for similaritymeasures: filename=similaritymeasures-1.1.0-py3-none-any.whl size=15430 sha256=e919991697e68e9c8240ba30e206fe2e55cf47f1d4ea88d356564d42c5fccb41
  Stored in directory: /root/.cache/pip/wheels/f4/48/ec/2126cdd1eb114d2c826f4ee014c6e80efe5f0095014d338bfc
Successfully built similaritymeasures


In [ ]:
def Avrage_S_D_T_Fv(g3):
    data=g3

    ################################################################# mohasebe Avrage D,T,Speed,Angel dar mode hamlonaghl

    g3['A_distance']=0.000000000000000000
    g3['A_Time']=0.000000000000000000
    g3['A_speed']=0.000000000000000000
    g3['Fv']=0.0000000000000000

    for k, n in data.groupby(['label']):
        mmm=0

        sumSpeed=n['Speed'].sum()
        sumDistance=n['DelD'].sum()
        sumTime=n['DelT'].sum()


        pointNum=len(n)
        mmm=n.iloc[0]['label']


        AvDist=sumDistance/(pointNum-1)
        AvTime=sumTime/(pointNum-1)

        AvSpeed1=sumSpeed/(pointNum-1)
        if math.isnan(AvSpeed1):
              AvSpeed1=0.000000000001
        g3.loc[(g3['label'] == mmm) , 'A_speed'] = AvSpeed1


        g3.loc[(g3['label'] == mmm) , 'A_distance'] = AvDist
        g3.loc[(g3['label'] == mmm) , 'A_Time'] = AvTime


    ############################################################################# mohasebe Fv dar mode hamlonaghl
        lamda1=AvSpeed1/4
        lamda2=AvSpeed1
        sigma=(lamda1+lamda2)/2

        op0=(g3['label']==mmm) & (g3['Speed'] >= 0) & (g3['Speed'] <= lamda1)
        op1=(g3['label']==mmm) & (g3['Speed'] > lamda1) & (g3['Speed'] <= sigma)
        op2=(g3['label']==mmm) & (g3['Speed'] > sigma) & (g3['Speed'] <= lamda2)


        g3.loc[op0, 'Fv']=1
        if AvSpeed1==0:
            AvSpeed1=0.000000000000000001
        g3.loc[op1, 'Fv']=1-(2*(((g3.loc[op1, 'Speed']-lamda1)/(lamda2-lamda1))**2))
        g3.loc[op2, 'Fv']=2*(((g3.loc[op2, 'Speed']-lamda2)/(lamda2-lamda1))**2)
        g3.loc[(g3['label']==mmm) & (g3['Speed'] > lamda2), 'Fv']=0


    return g3



def FAngel_FComplex_MoveAbil(g3):
    data=g3
    g3['Angel']= 0.000000000000000000000000
    g3['MA']=0.000000000000000000
    g3['Complex']=0.000000000000000000000000000000

    g3['FAngel']=1.0000
    g3['FMA']=0.000000000000000000
    g3['FComp']=0.00000000000000000000000000000000


    j=1
    pointNumber=len(g3)
    latitude=list(g3['lat']);longitude=list(g3['lon']);
    while j+1<pointNumber:

        a=GetDistance(longitude[j-1],latitude[j-1],longitude[j],latitude[j])
        b=GetDistance(longitude[j],latitude[j],longitude[j+1],latitude[j+1])
        c=GetDistance(longitude[j-1],latitude[j-1],longitude[j+1],latitude[j+1])

        p1 = np.array([longitude[j-1],latitude[j-1]])
        p2 = np.array([longitude[j+1],latitude[j+1]])
        p3 = np.array([longitude[j],latitude[j]])

        if (a!=0) and (b!=0) and (g3['label'][j+1]==g3['label'][j]):
            aa=(c**2 - b**2 - a**2)
            bb=(-2.0 * a * b)

            vv=aa/bb
            if vv>1:
                vv=1
            if vv<-1:
                vv=-1
            zz=math.acos(vv)
            #print("vv=",vv)
            #print("zz=",math.degrees(zz) )
        ################################################################### Angel
            g3['Angel'][j]=math.degrees(zz)

        ################################################################### Move Abolity
            g3['MA'][j] = (c/(a+b)) #stop ability

        #################################################################### Complexity
        norm = np.linalg.norm
        ll=norm(p2-p1)
        if ll==0:
            ll=0.000000000000000000001
        h0 = perpendicularDistance(longitude[j],latitude[j], longitude[j-1],latitude[j-1], longitude[j+1],latitude[j+1])
        #print('h0=',h0)
        h=np.abs(norm(np.cross(p2-p1, p1-p3)))/ll
        #print('h=',h)

        if c==0:
            c=0.000000000000000000001
        #print('c=',c)
        com=h0/c
        g3['Complex'][j] = com
        if g3['label'][j+1]!=g3['label'][j] or j==0 or j==pointNumber:
          g3['FComp'][j] =1
          g3['FComp'][j+1] =1

        j=j+1




    for k, n in data.groupby(['label']):
        ################################################################### FMoveAbility
        mmm=0
        minIndex=n['index'].min()
        maxIndex=n['index'].max()
        pointNum=len(n)

        sumMA=n['MA'].sum()
        AvMA=sumMA/pointNum

        mmm=n.iloc[0]['label']

        eta1=AvMA/4
        eta2=AvMA
        Mu=(eta1+eta2)/2
        op1=(g3['label']==mmm) & (g3['MA'] > eta1) & (g3['MA'] <= Mu)
        op2=(g3['label']==mmm) & (g3['MA'] > Mu) & (g3['MA'] <= eta2)


        g3.loc[(g3['label']==mmm) & (g3['MA'] >= 0) & (g3['MA'] <= eta1), 'FMA']=1
        if AvMA==0:
            AvMA=0.000000000000000001
        g3.loc[op1, 'FMA']=1-(2*(((g3.loc[op1, 'MA']-eta1)/(eta2-eta1))**2))
        g3.loc[op2, 'FMA']=2*((((g3.loc[op2, 'MA'])-eta2)/(eta2-eta1))**2)
        g3.loc[(g3['label']==mmm) & (g3['MA'] > eta2), 'FMA']=0

        ################################################################### NormComplexity
        maxComplex=n['Complex'].max()
        minComplex=n['Complex'].min()

        if maxComplex==0:
            maxComplex=0.000000000000000001

        g3.loc[(g3['label']==mmm) , 'FComp']=(((g3.loc[(g3['label']==mmm), 'Complex'])-minComplex)/(maxComplex-minComplex))
        g3.loc[minIndex , 'FComp']=1
        g3.loc[maxIndex , 'FComp']=1

        ####################################################################### FAngel

        sumAngel=n['Angel'].sum()
        AvAngel=sumAngel/pointNum

        teta1=AvAngel/4
        teta2=AvAngel
        gama=(teta1+teta2)/2
        op13=(g3['label']==mmm) & (g3['Angel'] > teta1) & (g3['Angel'] <= gama)
        op23=(g3['label']==mmm) & (g3['Angel'] > gama) & (g3['Angel'] <= teta2)


        g3.loc[(g3['label']==mmm) & (g3['Angel'] >= 0) & (g3['Angel'] <= teta1), 'FAngel']=1
        if AvAngel==0:
            AvAngel=0.000000000000000001
        g3.loc[op13, 'FAngel']=1-(2*(((g3.loc[op13, 'Angel']-teta1)/(teta2-teta1))**2))
        g3.loc[op23, 'FAngel']=2*(((g3.loc[op23, 'Angel']-teta2)/(teta2-teta1))**2)

        g3.loc[(g3['label']==mmm) & (g3['Angel'] > teta2), 'FAngel']=0





        ######******************************************FComplex***********************************************************
        #g3.loc[op1 , 'FComplex']=((g3.loc[op1, 'Complex']-minComplex)/(maxComplex-minComplex))
        ######**************************************FAngel*****************************************************************
        #g3.loc[op1 , '1-FAngel1']=1-((g3.loc[op1, 'Angel']-minAngel)/(maxAngel-minAngel))
        ######*************************************************************************************************************


    return g3

#---------------------------------------
def perpendicularDistance(lon0,lat0, lon1,lat1, lon2,lat2):

  [x0, y0,z0,s0] = utm.from_latlon(lat0,lon0)
  [x1, y1,z1,s1] = utm.from_latlon(lat1,lon1)
  [x2, y2,z2,s2] = utm.from_latlon(lat2,lon2)

  ll=np.sqrt(((x2-x1)*(x2-x1))+((y2-y1)*(y2-y1)))
  mm=abs(((x2-x1)*(y1-y0))-((x1-x0)*(y2-y1)))

  if ll==0:
    ll=0.000000000000000000001
  h=mm/ll


  return h


##-----------------------------------------------

import geopy.distance
def GetDistance1(lon1,lat1,lon2,lat2,h,m,s):


    coords_1 = (lat1,lon1)
    coords_2 = (lat2,lon2)

    dist= (geopy.distance.geodesic(coords_1, coords_2).km)*1000
    return dist

def GetDistance(lon1,lat1,lon2,lat2):
    EARTH_RADIUS=6373.5

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * math.asin(sqrt(a))

    distance = EARTH_RADIUS * c *1000

    return distance

def ComputMeanCoord(data,i,j):
    latitude=list(data['lat'])
    longitude=list(data['lon'])
    amount=j-i+1
    if amount==0:
        return float(latitude[i]),float(longitude[i])
    sumLongitude=0
    sumLatitude=0
    for k in range(i,j+1):
        sumLatitude+=float(latitude[k])
        sumLongitude+=float(longitude[k])
    return sumLatitude/amount,sumLongitude/amount



def checkSpeed(g3):
   for k, nn in g3.groupby(['Time']):
          sumT=0
          sumD=0

          sumT=nn['DelT'].sum()
          sumD=nn['DelD'].sum()
          time=nn.iloc[0]['Time']
          len8=len(nn)

          if sumT == 0.0:
            speed=(sumD/(1/len8))

            g3.loc[g3['Time']==time,'Speed']=speed
          else:
            speed=(sumD/sumT)

            g3.loc[g3['Time']==time,'Speed']=speed


   return g3

In [ ]:
from pandas.core.ops.mask_ops import kleene_or
from zmq.constants import THREAD_NAME_PREFIX
from trackintel.geogr import point_haversine_dist
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Point
import time
import glob
import os.path
import datetime
import os
import math
from math import sin, cos, sqrt, atan2, radians
import re
import copy
import urllib3
import warnings
warnings.filterwarnings("ignore")
import fiona
from functools import partial
import pyproj
from shapely.ops import transform
from pyproj import Proj, transform

import folium
from shapely import geometry
import utm
import tensorflow as tf


import alphashape
#from descartes import PolygonPatch
import folium


from geopy.geocoders import Nominatim
from ipywidgets import interact, fixed, widgets
import matplotlib.pyplot as plt
import networkx as nx
import osmnx as ox

from tqdm import tqdm



distThreh=100
timeThreh=100
 ########################################################################## Read Files and users #######################################################################################


def read_csv(csv_file):

    points = pd.read_csv(csv_file, skiprows=1, header=None, infer_datetime_format=True,parse_dates=[[3,4]])
    file_name=os.path.basename(csv_file)
    #points.columns = ['Dates_Time','lat', 'lon', 'label', 'user','checkPoint']
    points.columns = ['Dates_Time','lat', 'lon', 'label', 'Weekday','user','CheckStop','FID']
    return points , csv_file



def read_user(user_folder,i,sf):


    csv_files = glob.glob(os.path.join(user_folder, '*.csv'))
    print(csv_files)
    testing=[]
    All=[]
    gfinal_snum=0
    gtest_snum=0

    for j,f in enumerate(csv_files):
        print('\n')

        print('----------------- [%d/%d] processing file %s----------------' % (j+1, len(csv_files), f))

        [qq,pp]=read_csv(f)

        dddddd=(pd.to_datetime(qq['Dates_Time']).dt.date)[0]


        directory='/content/drive/My Drive/mir/final/2/OutPut/data/%s/Simin_Output_%s_%s.csv' % (sf,sf,dddddd)
        if os.path.isfile(directory):
            continue
        print("no")
        PointOneDay=[qq]
        PointsOfOneDay = pd.concat(PointOneDay)

        date_time,dfG=Stop_Point(PointsOfOneDay,sf)

        All=All+[dfG]
    if All!=[]:
      All1=pd.concat(All)
    else:
      All1=[]
    #print(All1)

    return All1


def read_all_users(folder):
    gfinal_Sum=0
    gtest_Sum=0
    subfolders = os.listdir(folder)
    dfs_test = []
    dfs_All = []
    for i, sf in enumerate(subfolders):
        print('++++++++++++++++++ [%d/%d] processing user %s+++++++++++++++++++++++++++++++++++++++' % (i+1, len(subfolders), sf))
        All=read_user(os.path.join(folder,sf),i,sf)

        dfs_All=dfs_All+[All]
    if dfs_All1!= []:
      dfs_All1=pd.concat(dfs_All)
        #dfs_All.append(All)

    print(gfinal_Sum)
    print(gtest_Sum)

    #pd.concat(dfs_All)
    print("min_lat= ",dfs_All1['lat'].min())
    print("min_lon= ",dfs_All1['lon'].min())
    print("max_lat= ",dfs_All1['lat'].max())
    print("max_lon= ",dfs_All1['lon'].max())
    print(dfs_All1)


    return  gfinal_Sum,gtest_Sum,dfs_test,dfs_All

########################################################################## tabe mohasebe daraje stop point boudan ################################################################################

def Stop_Point(PointsOfOneDay,sf):
        #print("1-Starttttttt")
        start = time.process_time()
        #print("time =  ",time.process_time() - start)
        pd.set_option('display.max_columns', None)
        #pd.set_option('display.max_rows', None)
        PointsOfOneDay['Dates'] = pd.to_datetime(PointsOfOneDay['Dates_Time']).dt.date
        PointsOfOneDay['Time'] = pd.to_datetime(PointsOfOneDay['Dates_Time']).dt.time
        PointsOfOneDay.drop(inplace=True, columns='Dates_Time')





        g3=PointsOfOneDay
        #g3=g2[(g2['lat']>= 21.5) & (g2['lat']<= 52) & (g2['lon']>= 75.97) & (g2['lon']<= 122)] # joda kardan mahdode chin

        Date=g3.iloc[0]['Dates']
        date_time = Date.strftime("%Y-%m-%d")

        g3['index'] = 0
        g3['year'] = g3['Dates'].apply(lambda x : x.year)
        g3['month'] = g3['Dates'].apply(lambda x : x.month)
        g3['day'] = g3['Dates'].apply(lambda x : x.day)
        g3['hour']=g3['Time'].apply(lambda x : x.hour)
        g3['minute']=g3['Time'].apply(lambda x : x.minute)
        g3['second']=g3['Time'].apply(lambda x : x.second)

        pointNumber=len(g3)
        latitude=list(g3['lat']);longitude=list(g3['lon']);day=list(g3['day']);hour=list(g3['hour']);minute=list(g3['minute']);second=list(g3['second'])

        ############################################################## mohasebe DelT, DelD, Speed for each point ################################################################################

        g3['DelT']=0.0000000000000000
        g3['DelD']=0.00000000000000
        #g3['DelD1']=0.00000000000000
        g3['Speed']=0.000000000000000000
        g3['StopPoint']=0
        j=0
        '''print('2-mohasebe DelT, DelD, Speed for each point')
        print("time =  ",time.process_time() - start)'''
        while j+1<pointNumber:

            t=abs(86400*(g3['day'].iloc[j+1]-g3['day'].iloc[j])+3600*(g3['hour'].iloc[j+1]-g3['hour'].iloc[j])+60*(g3['minute'].iloc[j+1]-g3['minute'].iloc[j])+(g3['second'].iloc[j+1]-g3['second'].iloc[j]))

            d=point_haversine_dist(g3['lon'].iloc[j],g3['lat'].iloc[j],g3['lon'].iloc[j+1],g3['lat'].iloc[j+1])


            g3['index'][j+1]=j+1
            g3['DelD'][j+1]=d

            g3['DelT'][j+1]=t
            if t==0:
              g3['Speed'][j+1]=0

              g3['DelT'][j+1]=0.00000

            else:
              g3['Speed'][j+1]=d/t


              if g3['label'][j+1]!=g3['label'][j]:
                g3['Speed'][j+1]=0.0000000
                g3['DelD'][j+1]=0.0000000

                g3['DelT'][j+1]=0.000000
                g3['StopPoint'][j+1]=1

              if ((g3['label'][j+1]=='walk') or (g3['label'][j+1]=='run')or (g3['label'][j+1]=='bus') or (g3['label'][j+1]=='taxi'))and ((g3['DelD'][j+1]>=200)or(g3['DelT'][j+1]>=100)):

                g3['DelT'][j+1]=0.0000000
                g3['Speed'][j+1]=0.0000000
                g3['DelD'][j+1]=0.0000000

              if ((g3['label'][j+1]=='car') or (g3['label'][j+1]=='train')) and ((g3['DelD'][j+1]>=30000)or(g3['DelT'][j+1]>=400) ):

                g3['DelD'][j]=0.0000000
                g3['DelT'][j+1]=0.0000000
                g3['Speed'][j+1]=0.0000000

              if math.isnan(g3['DelD'][j+1]):
                g3['DelD'][j+1]=0.00000000

              if (g3['Speed'][j+1]>=10000):
                g3['DelD'][j+1]=0.0000
                g3['Speed'][j+1]=0.0000


            #print("g3['DelT'][j+1]=",(g3['DelT'][j+1]))

            j=j+1


        g3=checkSpeed(g3)




############################################################# mohasebe Avrage_S_D_T_Fv ################################################################################
        '''print("3-mohasebe Avrage_S_D_T_Fv")
        print("time =  ",time.process_time() - start)'''
        g3=Avrage_S_D_T_Fv(g3)
        #print(g3)

        ############################################################## mohasebe FAngel_FComplex_MoveAbil ################################################################################
        '''print("4-mohasebe FAngel_FComplex_MoveAbil")
        print("time =  ",time.process_time() - start)'''
        g3=FAngel_FComplex_MoveAbil(g3)

        ############################################################## mohasebe Physical stop point ################################################################################
        '''print("5-mohasebe Physical stop point")
        print("time =  ",time.process_time() - start)'''


        k=0
        SP=[]
        l=0
        distThreh=0
        coff=0
        coffC=0

        coffT=0
        coffTC=0
        while k<pointNumber-1:
                l=k+1
                Token=0
                while l<pointNumber:
                    dist=GetDistance(longitude[k],latitude[k],longitude[l],latitude[l])
                    labell= g3['label'].iloc[k]
                    if labell=='walk':
                      coff=30
                      coffC=30
                    elif labell=='run':
                      coff=20
                      coffC=20
                    elif labell=='bus':
                      coff=14
                      coffC=50
                    elif labell=='train':
                      coff=10
                      coffC=100
                    elif labell=='car':
                      coff=6
                      coffC=25
                    elif labell=='taxi':
                      coff=8
                      coffC=20
                    elif labell=='bike':
                      coff=14
                      coffC=30
                    elif labell=='motocycle':
                      coff=10
                      coffC=40
                    elif labell=='boat':
                      coff=8
                      coffC=500
                    elif labell=='airplane':
                      coff=1.6
                      coffC=1000
                    else:
                      coff=8
                      coffC=30
                    distThreh=(g3['A_speed'].iloc[k])*coff
                    #distThreh=coffC
                    if dist<float(distThreh):
                        l=l+1
                        Token=1
                    else:
                        l=l+1
                        break
                Time=abs(86400*(day[l-1]-day[k])+3600*(hour[l-1]-hour[k])+60*(minute[l-1]-minute[k])+(second[l-1]-second[k]))
                if labell=='walk':
                      coffT=50
                      coffTC=100
                elif labell=='run':
                      coffT=15
                      coffTC=50
                elif labell=='bus':
                      coffT=2
                      coffTC=40
                elif labell=='train':
                      coffT=5
                      coffTC=150
                elif labell=='car':
                      coffT=8
                      coffTC=100
                elif labell=='taxi':
                      coffT=4
                      coffTC=50
                elif labell=='bike':
                      coffT=11
                      coffTC=100
                elif labell=='motocycle':
                      coffT=12
                      coffTC=100
                elif labell=='boat':
                      coffT=10
                      coffTC=500
                elif labell=='airplane':
                      coffT=1
                      coffTC=1000
                else:
                      coffT=.5
                if  g3['A_Time'].iloc[k]<3:
                  timeThreh=(g3['A_speed'].iloc[k])*coffT
                  #timeThreh=coffTC
                else :
                  timeThreh=(g3['A_Time'].iloc[k])*coffT
                  #timeThreh=coffTC
                if Time>float(timeThreh):
                    g3['StopPoint'][k:l-1]=1
                    S_latitude,S_longitude=ComputMeanCoord(g3,k,l-1)
                    SarvT=str(g3['year'].iloc[k])+'-'+str(g3['month'].iloc[k])+'-'+str(g3['day'].iloc[k])+' '+str(g3['hour'].iloc[k])+':'+str(g3['minute'].iloc[k])+':'+str(g3['second'].iloc[k])
                    SlevT=str(g3['year'].iloc[l-1])+'-'+str(g3['month'].iloc[l-1])+'-'+str(g3['day'].iloc[l-1])+' '+str(g3['hour'].iloc[l-1])+':'+str(g3['minute'].iloc[l-1])+':'+str(g3['second'].iloc[l-1])
                    SP.append([g3['user'].iloc[k],S_longitude,S_latitude,SarvT,SlevT,[k,l-1],g3['label'].iloc[k]])

                    k=l-1

                k=k+1

        g3=g3.drop(['year','month','day','hour','minute','second'],axis=1)



############################################################## mohasebe Contextual stop point ################################################################################
        '''print('6-mohasebe Contextual stop point')
        print("time =  ",time.process_time() - start)'''
        g3['Buff_dist']=0
        g3.loc[g3['label']=='walk','Buff_dist']=15*(g3.loc[g3['label']=='walk','A_speed'] )
        g3.loc[g3['label']=='run','Buff_dist']=10*(g3.loc[g3['label']=='run','A_speed'] )
        g3.loc[g3['label']=='bus','Buff_dist']=9*(g3.loc[g3['label']=='bus','A_speed'] )
        g3.loc[g3['label']=='train','Buff_dist']=5*(g3.loc[g3['label']=='train','A_speed'] )
        g3.loc[g3['label']=='car','Buff_dist']=3*(g3.loc[g3['label']=='car','A_speed'] )
        g3.loc[g3['label']=='taxi','Buff_dist']=4*(g3.loc[g3['label']=='taxi','A_speed'] )
        g3.loc[g3['label']=='bike','Buff_dist']=7*(g3.loc[g3['label']=='bike','A_speed'] )
        g3.loc[g3['label']=='motocycle','Buff_dist']=5*(g3.loc[g3['label']=='motocycle','A_speed'] )
        g3.loc[g3['label']=='boat','Buff_dist']=4*(g3.loc[g3['label']=='boat','A_speed'] )
        g3.loc[g3['label']=='airplane','Buff_dist']=0.8*(g3.loc[g3['label']=='airplane','A_speed'] )


        #print(g3)
        #---------------------------------------------------point in poly ------POLYGON------------
        g3['coords'] = list(zip(g3['lon'],g3['lat']))
        g3['coords'] = g3['coords'].apply(Point)

        g3["PointInPoly"]=0
        g3["PointInPoly1"]=0
        g3['fclass2']=0
        g3['fclass3']=0

        print('g3============',g3.shape)

        print('building============',building.shape);

        points11 = gpd.GeoDataFrame(g3, geometry='coords', crs=building.crs)
        print('points11===========',points11.shape);



        # Perform spatial join to match points and polygons
        pointInPolys = gpd.tools.sjoin(points11, building, op="intersects", how='left')
        print('pointInPolys===========',pointInPolys.shape);
        #print('pointInPolys===========',pointInPolys.head)

        pointInPolys=pointInPolys.drop(['index_right'],axis=1)
        print('pointInPolys===========',pointInPolys.shape);


        Cond0=((pointInPolys['label']=='boat')& ((pointInPolys['fclass'] == 'transportation')|(pointInPolys['fclass'] == 'marina')|(pointInPolys['fclass'] == 'dock')|(pointInPolys['fclass'] == 'boat_sharing') | (pointInPolys['fclass'] =='boat_rental') | (pointInPolys['fclass'] =='pier') | (pointInPolys['fclass'] =='ferry_terminal')))
        Cond1=((pointInPolys['label']=='train')& ((pointInPolys['fclass'] == 'transportation')|(pointInPolys['fclass'] == 'train_station' ) |(pointInPolys['fclass'] == 'railway_halt') | (pointInPolys['fclass'] =='railway_station') | (pointInPolys['fclass'] =='stop') | (pointInPolys['fclass'] =='tram stop')))
        Cond2=((pointInPolys['label']=='bus')& ((pointInPolys['fclass'] == 'transportation')|(pointInPolys['fclass'] == 'garage')|(pointInPolys['fclass'] == 'garages')|(pointInPolys['fclass'] == 'service')|(pointInPolys['fclass'] == 'bus_stop') | (pointInPolys['fclass'] =='fuel') | (pointInPolys['fclass'] =='bus_station')| (pointInPolys['fclass'] =='crossing') | (pointInPolys['fclass'] =='motorway junction')|(pointInPolys['fclass'] =='parking_underground')| (pointInPolys['fclass'] =='parking')|(pointInPolys['fclass'] =='wayside_cross')|(pointInPolys['fclass'] =='stop')))
        Cond3=((pointInPolys['label']=='bike')& ((pointInPolys['fclass'] == 'service')|(pointInPolys['fclass'] == 'bicycle_parking') | (pointInPolys['fclass'] =='bicycle_repair_station') | (pointInPolys['fclass'] =='bicycle_rental')| (pointInPolys['fclass'] =='crossing') | (pointInPolys['fclass'] =='parking_bicycle')|(pointInPolys['fclass'] =='bicycle_shop')| (pointInPolys['fclass'] =='parking')|(pointInPolys['fclass'] =='wayside_cross')|(pointInPolys['fclass'] =='stop')))
        Cond4=((pointInPolys['label']=='motorcycle')& ((pointInPolys['fclass'] == 'service')|(pointInPolys['fclass'] == 'parking_underground') | (pointInPolys['fclass'] =='motorcycle_parking') | (pointInPolys['fclass'] =='fuel')| (pointInPolys['fclass'] =='crossing') | (pointInPolys['fclass'] =='parking')|(pointInPolys['fclass'] =='wayside_cross')|(pointInPolys['fclass'] =='stop')))
        Cond5=((pointInPolys['label']=='taxi')& ((pointInPolys['fclass'] == 'transportation')|(pointInPolys['fclass'] == 'garage')|(pointInPolys['fclass'] == 'garages')|(pointInPolys['fclass'] == 'service')|(pointInPolys['fclass'] == 'parking_underground') | (pointInPolys['fclass'] =='taxi') | (pointInPolys['fclass'] =='fuel')| (pointInPolys['fclass'] =='crossing')| (pointInPolys['fclass'] =='motorway junction') | (pointInPolys['fclass'] =='parking')|(pointInPolys['fclass'] =='wayside_cross')|(pointInPolys['fclass'] =='stop')))
        Cond6=((pointInPolys['label']=='airplane')& ((pointInPolys['fclass'] == 'transportation')|(pointInPolys['fclass'] == 'airfield') | (pointInPolys['fclass'] =='airport') | (pointInPolys['fclass'] =='helipad') | (pointInPolys['fclass'] =='alpine_hut')))
        Cond7=((pointInPolys['label']=='car')& ((pointInPolys['fclass'] == 'transportation')|(pointInPolys['fclass'] == 'garage')|(pointInPolys['fclass'] == 'garages')|(pointInPolys['fclass'] == 'carport')|(pointInPolys['fclass'] == 'service')|(pointInPolys['fclass'] == 'car_rental') | (pointInPolys['fclass'] =='car_sharing') | (pointInPolys['fclass'] =='car_wash')| (pointInPolys['fclass'] =='crossing') | (pointInPolys['fclass'] =='vehicle_inspection')|(pointInPolys['fclass'] =='fuel')| (pointInPolys['fclass'] =='parking')|(pointInPolys['fclass'] =='wayside_cross')|(pointInPolys['fclass'] =='stop') |(pointInPolys['fclass'] =='parking_entrance')|(pointInPolys['fclass'] =='caravan_site')|(pointInPolys['fclass'] =='motorway junction')|(pointInPolys['fclass'] =='parking_multistorey')|(pointInPolys['fclass'] =='stop')|(pointInPolys['fclass'] =='car_dealership')|(pointInPolys['fclass'] =='parking_underground')|(pointInPolys['fclass'] =='parking_space')))
        Cond8=((pointInPolys['label']=='walk')|(pointInPolys['label']=='run'))

        pointInPolys.loc[(pointInPolys["fclass"].notnull()) & (Cond0 |Cond1|Cond2|Cond3|Cond4|Cond5|Cond6|Cond7|Cond8) , 'PointInPoly']=1
        pointInPolys = pointInPolys[~pointInPolys.index.duplicated(keep='first')]
        print('pointInPolys++++===========',pointInPolys.shape);
        FID_PP=pointInPolys.index[pointInPolys['PointInPoly'] == 1].tolist()

        print("FID_PP=",FID_PP)
        g3.loc[FID_PP,'PointInPoly']=1
        g3['fclass3']=pointInPolys['fclass']
        g3.loc[FID_PP,'fclass2']=pointInPolys.loc[FID_PP,'fclass']

        g3.loc[FID_PP,'PointInPoly1']=1


        #------------------------------------------------------------------point in poly ------Point----------------------------------

        test= gpd.GeoDataFrame(g3,geometry='coords', crs=building.crs)

        test['buffer']=0
        print('test===========',test.shape);
        #print('test===========',test.head)
        nnnn=len(test)
        #print(test)
        test=test.to_crs(epsg=7855)

        print('test===7855=====',test.shape);
        #print('test===7855=====',test.head)
        #print(test)
        kk=0

        while kk<nnnn:
          if (test['PointInPoly'].iloc[kk]) !=1:
            buf=(test['Buff_dist'].iloc[kk])
            test['buffer'][kk]=(test['coords'].iloc[kk]).buffer(buf)
          kk=kk+1

        print('test===buf=====',test.shape);




        BOB=test.loc[(test["PointInPoly"]!=1)]


        Buffers = gpd.GeoDataFrame(BOB, geometry='buffer',crs="EPSG:7855")
        print('Buffers===buf=====',Buffers.shape);


        Buffers=Buffers.to_crs(epsg=4326)
        print('Buffers===buf=====',Buffers.shape);


        pointInPolys2 = gpd.tools.sjoin(Buffers,pois, op="intersects",how='left')
        #pointInPolys2.to_csv('/content/drive/My Drive/mir/final/Simin_%s.csv' % (date_time) , index = False)
        print('pointInPolys2===buf=====',pointInPolys2.shape);

        Con0=((pointInPolys2['label']=='boat')& ((pointInPolys2['fclass'] == 'transportation')|(pointInPolys2['fclass'] == 'marina')|(pointInPolys2['fclass'] == 'boat_sharing') | (pointInPolys2['fclass'] =='boat_rental') | (pointInPolys2['fclass'] =='pier') | (pointInPolys2['fclass'] =='ferry_terminal')))
        Con1=((pointInPolys2['label']=='train')& ((pointInPolys2['fclass'] == 'transportation')|(pointInPolys2['fclass'] == 'train_station' ) |(pointInPolys2['fclass'] == 'railway_halt') | (pointInPolys2['fclass'] =='railway_station') | (pointInPolys2['fclass'] =='stop') | (pointInPolys2['fclass'] =='tram stop')))
        Con2=((pointInPolys2['label']=='bus')& ((pointInPolys2['fclass'] == 'transportation')|(pointInPolys2['fclass'] == 'bus_stop')|(pointInPolys2['fclass'] == 'garages')|(pointInPolys2['fclass'] == 'service') |(pointInPolys2['fclass'] == 'bus_stop') | (pointInPolys2['fclass'] =='fuel') | (pointInPolys2['fclass'] =='bus_station')| (pointInPolys2['fclass'] =='crossing') | (pointInPolys2['fclass'] =='motorway junction')|(pointInPolys2['fclass'] =='parking_underground')| (pointInPolys2['fclass'] =='parking')|(pointInPolys2['fclass'] =='wayside_cross')|(pointInPolys2['fclass'] =='stop')))
        Con3=((pointInPolys2['label']=='bike')& ((pointInPolys2['fclass'] == 'service') |(pointInPolys2['fclass'] == 'bicycle_parking') | (pointInPolys2['fclass'] =='bicycle_repair_station') | (pointInPolys2['fclass'] =='bicycle_rental')| (pointInPolys2['fclass'] =='crossing') | (pointInPolys2['fclass'] =='parking_bicycle')|(pointInPolys2['fclass'] =='bicycle_shop')| (pointInPolys2['fclass'] =='parking')|(pointInPolys2['fclass'] =='wayside_cross')|(pointInPolys2['fclass'] =='stop')))
        Con4=((pointInPolys2['label']=='motorcycle')& ((pointInPolys2['fclass'] == 'service') |(pointInPolys2['fclass'] == 'parking_underground') | (pointInPolys2['fclass'] =='motorcycle_parking') | (pointInPolys2['fclass'] =='fuel')| (pointInPolys2['fclass'] =='crossing') | (pointInPolys2['fclass'] =='parking')|(pointInPolys2['fclass'] =='wayside_cross')|(pointInPolys2['fclass'] =='stop')))
        Con5=((pointInPolys2['label']=='taxi')& ((pointInPolys2['fclass'] == 'transportation')|(pointInPolys2['fclass'] == 'garage')|(pointInPolys2['fclass'] == 'garages')|(pointInPolys2['fclass'] == 'service') |(pointInPolys2['fclass'] == 'parking_underground') | (pointInPolys2['fclass'] =='taxi') | (pointInPolys2['fclass'] =='fuel')| (pointInPolys2['fclass'] =='crossing')| (pointInPolys2['fclass'] =='motorway junction') | (pointInPolys2['fclass'] =='parking')|(pointInPolys2['fclass'] =='wayside_cross')|(pointInPolys2['fclass'] =='stop')))
        Con6=((pointInPolys2['label']=='airplane')& ((pointInPolys2['fclass'] == 'transportation')|(pointInPolys2['fclass'] == 'airfield') | (pointInPolys2['fclass'] =='airport') | (pointInPolys2['fclass'] =='helipad') | (pointInPolys2['fclass'] =='alpine_hut')))
        Con7=((pointInPolys2['label']=='car')& ((pointInPolys2['fclass'] == 'transportation')|(pointInPolys2['fclass'] == 'garage')|(pointInPolys2['fclass'] == 'garages')|(pointInPolys2['fclass'] == 'carport')|(pointInPolys2['fclass'] == 'service') |(pointInPolys2['fclass'] == 'car_rental') | (pointInPolys2['fclass'] =='car_sharing') | (pointInPolys2['fclass'] =='car_wash')| (pointInPolys2['fclass'] =='crossing') | (pointInPolys2['fclass'] =='vehicle_inspection')|(pointInPolys2['fclass'] =='fuel')| (pointInPolys2['fclass'] =='parking')|(pointInPolys2['fclass'] =='wayside_cross')|(pointInPolys2['fclass'] =='stop') |(pointInPolys2['fclass'] =='parking_entrance')|(pointInPolys2['fclass'] =='caravan_site')|(pointInPolys2['fclass'] =='motorway junction')|(pointInPolys2['fclass'] =='parking_multistorey')|(pointInPolys2['fclass'] =='stop')|(pointInPolys2['fclass'] =='car_dealership')|(pointInPolys2['fclass'] =='parking_underground')|(pointInPolys2['fclass'] =='parking_space')))
        Con8=((pointInPolys2['label']=='walk')|(pointInPolys2['label']=='run'))


        pointInPolys2.loc[(pointInPolys2["fclass"].notnull()) & (Con0 |Con1|Con2|Con3|Con4|Con5|Con6|Con7|Con8) , 'PointInPoly']=2
        #print(pointInPolys2[pointInPolys2['PointInPoly']==2])
        pointInPolys22 = pointInPolys2.loc[pointInPolys2["PointInPoly"] == 2 ]

        print('pointInPolys22===tagcheked=====',pointInPolys22.shape);



        pointInPolys22['pop']=0
        pointInPolys22['pop1']=0
        nM=len(pointInPolys22)
        jj=0
        '''print('18-check for distance of semantic pois')
        print("time =  ",time.process_time() - start)'''
        '''pointInPolys22['distpoi']=0
        pointInPolys22['distpoi']=GetDistance(pointInPolys22['lon'],pointInPolys22['lat'],((pois.loc[pois['osm_id']==(pointInPolys22['osm_id']),'geometry']).iloc).x,((pois.loc[pois['osm_id']==(pointInPolys22['osm_id']),'geometry']).iloc).y)
        '''

        while jj<nM:
          if (pointInPolys22['PointInPoly'].iloc[jj]) ==2:

            TraX=pointInPolys22['lon'].iloc[jj]
            TraY=pointInPolys22['lat'].iloc[jj]

            #print(TraX)
            #print(TraY)

            osm_id=pointInPolys22['osm_id'].iloc[jj]


            poiPoint=(pois.loc[pois['osm_id']==osm_id,'geometry']).iloc[0]
            POIx=poiPoint.x
            POIy=poiPoint.y

            distx=GetDistance(TraX,TraY,POIx,POIy)

            pointInPolys22['pop1'].iloc[jj]=1
            pointInPolys22['pop'].iloc[jj]=(1-(distx/(pointInPolys22['Buff_dist'].iloc[jj])))

          jj=jj+1

        print('pointInPolys22===check dist for all=====',pointInPolys22.shape);

        testttt=pointInPolys22

        for k, n in testttt.groupby(['FID']):

          mintoul=0
          mmm=n.iloc[0]['FID']

          mintoul=n['pop'].max()
          n.loc[:,'pop']=mintoul

          pointInPolys22.loc[(g3['FID']==mmm)]= n.loc[(n['pop']==mintoul)]

          #pointInPolys22.loc[(g3['FID']==mmm) , 'pop']= mintoul
        '''print('20-create final file')
        print("time =  ",time.process_time() - start)'''

        pointInPolys22 = pointInPolys22[~pointInPolys22.index.duplicated(keep='first')]
        #print('pointInPolys22===buf=====',pointInPolys22);
        print('pointInPolys22===dup=====',pointInPolys22.shape)
        #print(pointInPolys22)
        file=pointInPolys22[['buffer']]
        file=gpd.GeoDataFrame(file, geometry='buffer')
        file.crs= "+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs"


        FID_PP2=pointInPolys22.index[pointInPolys22['PointInPoly'] == 2].tolist()

        print("FID_PP2=",FID_PP2)
        g3.loc[FID_PP2,'PointInPoly']=pointInPolys22.loc[FID_PP2,'pop']
        g3.loc[FID_PP2,'fclass2']=pointInPolys22.loc[FID_PP2,'fclass']
        g3['fclass3']=pointInPolys22['fclass']
        g3.loc[FID_PP2,'PointInPoly1']=pointInPolys22.loc[FID_PP2,'pop1']
        print('g3===buf=====',g3.shape);
        g3=g3.drop(['coords'],axis=1)




        g3['WS11']=((2/15)*g3['StopPoint']+(3/15)*g3['PointInPoly']+(3/15)*g3['Fv']+(3/15)*g3['FAngel']+(2/15)*g3['FMA']+(2/15)*g3['FComp'])
        g3['WS11_2']=((2/15)*g3['StopPoint']+(3/15)*g3['PointInPoly1']+(3/15)*g3['Fv']+(3/15)*g3['FAngel']+(2/15)*g3['FMA']+(2/15)*g3['FComp'])
        g3['Stop_result']=g3['WS11_2'].apply(lambda x: 1 if x >= 0.5 else 0)
        g4=g3[['FID','user','lat','lon','label','Dates','Time','Weekday','DelD','DelT','Speed','A_speed','Angel','MA','Complex','StopPoint', 'Fv', 'FAngel', 'FMA' ,'FComp' ,'PointInPoly','PointInPoly1','fclass2', 'fclass3','CheckStop',"Stop_result",'WS11','WS11_2']]

        #print(g4.loc[g4["PointInPoly"]!=0])

        directory='/content/drive/My Drive/mir/final/2/OutPut/data/%s' % sf
        if not os.path.exists(directory):
            os.makedirs(directory)
        g4.to_csv('/content/drive/My Drive/mir/final/2/OutPut/data/%s/Simin_Output_%s_%s.csv' % (sf,sf,date_time) , index = False)
        #-------------------------------------------------------------------------------------
        #g3=g3.drop(['year','month','day','hour','minute','second'],axis=1)
        #
        #output=pd.DataFrame(SP,columns=['user','lon','lat','arriveTime','leftTime','scope','label'])
        #output.to_csv('w1\%s\%s+SP.csv' % (sf,date_time), index = False)
        #print(date_time)
        print('21-end')
        print("time =  ",time.process_time() - start)
        return date_time,g4




In [ ]:
import os
os.chdir(r"/content/drive/")
(gfinal_Sum,gtest_Sum,dfs_test,dfs_All)=read_all_users('/content/drive/My Drive/mir/final/2/DATA')

++++++++++++++++++ [1/4] processing user 052+++++++++++++++++++++++++++++++++++++++
['/content/drive/My Drive/mir/final/2/DATA/052/2008-05-31.csv', '/content/drive/My Drive/mir/final/2/DATA/052/2008-06-28.csv', '/content/drive/My Drive/mir/final/2/DATA/052/2008-06-20.csv', '/content/drive/My Drive/mir/final/2/DATA/052/2008-05-27.csv', '/content/drive/My Drive/mir/final/2/DATA/052/2008-06-09.csv', '/content/drive/My Drive/mir/final/2/DATA/052/2008-05-22.csv', '/content/drive/My Drive/mir/final/2/DATA/052/2008-06-04.csv', '/content/drive/My Drive/mir/final/2/DATA/052/2008-06-25.csv', '/content/drive/My Drive/mir/final/2/DATA/052/2008-05-26.csv', '/content/drive/My Drive/mir/final/2/DATA/052/2008-06-07.csv', '/content/drive/My Drive/mir/final/2/DATA/052/2008-06-13.csv', '/content/drive/My Drive/mir/final/2/DATA/052/2008-06-01.csv', '/content/drive/My Drive/mir/final/2/DATA/052/2008-06-16.csv', '/content/drive/My Drive/mir/final/2/DATA/052/2008-06-15.csv', '/content/drive/My Drive/mir/fina

UnboundLocalError: local variable 'dfs_All1' referenced before assignment

In [ ]:
gfinal_Sum,gtest_Sum,dfs_test,dfs_All=read_all_users('data')
m=pd.concat(dfs_All)
m.to_csv('OutPut/Simin_all_no_sort.csv' , index = False)
nnn=m.sort_values(by=["user", "Dates", "Time"], inplace=True)
m.to_csv('OutPut/Simin_all_sort.csv' , index = False)
g3=m

natayej=pd.DataFrame()

g3['WS11']=((2/15)*g3['StopPoint']+(3/15)*g3['PointInPoly']+(3/15)*g3['Fv']+(3/15)*g3['FAngel']+(2/15)*g3['FMA']+(2/15)*g3['FComp'])
g3['out']=0
out=g3
out.loc[ (out['WS11']>=0.5),'out']=1
for k, n in out.groupby(['label']):
  te=len(n)
  label=k
  TMove=0;TStop=0;FMove=0;FStop=0
  print('TMove=',TMove,'   TStop=',TStop,'    FMove=',FMove,'   FStop=',FStop)
  oon=0
  Overal_accuracyn=0
  Precision_Stopn=0
  Precision_Moven=0
  Recall_Stopn=0
  Recall_Moven=0
  f_score_Stopn=0
  f_score_Moven=0
  while oon<te:
      if ((n['out'].iloc[oon]==0) and (n['CheckStop'].iloc[oon]==0)):
            TMove=TMove+1
      if ((n['out'].iloc[oon]==1) and (n['CheckStop'].iloc[oon]==1)):
            TStop=TStop+1

      if ((n['out'].iloc[oon]==0) and (n['CheckStop'].iloc[oon]==1)):
            FMove=FMove+1

      if ((n['out'].iloc[oon]==1) and (n['CheckStop'].iloc[oon]==0)):
            FStop=FStop+1
      oon=oon+1
  print(label)
  print('TMove=',TMove,'   TStop=',TStop,'    FMove=',FMove,'   FStop=',FStop)
  if TMove==0:
    TMove=0.000000000000001
  if TStop==0:
    TStop=0.000000000000001
  print('--------------------------')
  Overal_accuracyn=((TStop+TMove)/(TStop+TMove+FStop+FMove))*100
  Precision_Stopn=(TStop/(TStop+FStop))*100
  Precision_Moven=(TMove/(TMove+FMove))*100
  Recall_Stopn=(TStop/(TStop+FMove))*100
  Recall_Moven=(TMove/(TMove+FStop))*100
  f_score_Stopn=2*((Precision_Stopn*Recall_Stopn)/(Precision_Stopn+Recall_Stopn))
  f_score_Moven=2*((Precision_Moven*Recall_Moven)/(Precision_Moven+Recall_Moven))
  natayej = natayej.append({'label' : label, 'Overal_accuracyn' : Overal_accuracyn, 'Precision_Stopn' : Precision_Stopn,
                            'Precision_Moven':Precision_Moven, 'Recall_Stopn':Recall_Stopn,'Recall_Moven':Recall_Moven,
                            'f_score_Stopn':f_score_Stopn,'f_score_Moven':f_score_Moven},ignore_index = True)

print(natayej)